# Librerias

In [3]:
#Importemos las librerías Necesarias:
import scipy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, time

In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.layers import *
from keras.models import *

***

# Main Code

Dado que En intentos anteriores se ha recibido un error, haremos lo siguiente:

In [5]:
# !nvcc --version 
# !nvidia-smi

# Veamos los dispositivos Fisicos de tipo GPU:
ph_dev = tf.config.experimental.list_physical_devices('GPU')
print(ph_dev)

[]


In [6]:
(X_train, y_train), (X_test, y_test) = tfds.as_numpy(
    tfds.load(
            name='rock_paper_scissors',
            split=['train', 'test'],
            batch_size=-1,
            as_supervised=True
        )
    )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteFTJIBB/rock_paper_scissors-train.tfrecord


  0%|          | 0/2520 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteFTJIBB/rock_paper_scissors-test.tfrecord


  0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [7]:
# Veamos a que nos enfrentamos:
X_train[0].shape, y_train.shape, y_test.shape

((300, 300, 3), (2520,), (372,))

In [8]:
df_train = pd.get_dummies(pd.Categorical(y_train))
df_test = pd.get_dummies(pd.Categorical(y_test))
y_train = df_train.to_numpy()
y_test = df_test.to_numpy()
y_train.shape, y_test.shape

((2520, 3), (372, 3))

De aquí sabemos que las imágenes tienen 300x300 pixeles, ademas como son imágenes en RGB cada una de ellas tiene una profundidad de 3 capas, de modo que cada una de ellas mostrará en el rango 0, 255 la intensidad del color en el pixel analizado.

In [9]:
'''# Reescalemos primero las imagenes a fin de ocupar menos memoria:
train, test = list(), list()
size = 120
for image in X_train:
    train.append(tf.image.resize(image, [size,size]))
for image in X_test:
    test.append(tf.image.resize(image, [size,size]))

X_test = np.array(train)
X_train = np.array(train)
    
X_test.shape, y_test.shape'''

'# Reescalemos primero las imagenes a fin de ocupar menos memoria:\ntrain, test = list(), list()\nsize = 120\nfor image in X_train:\n    train.append(tf.image.resize(image, [size,size]))\nfor image in X_test:\n    test.append(tf.image.resize(image, [size,size]))\n\nX_test = np.array(train)\nX_train = np.array(train)\n    \nX_test.shape, y_test.shape'

In [10]:
# escalemos los datos al rango 0, 1:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [11]:
# Creemos el modelo:
model = Sequential()
# Primera convolución
model.add(Conv2D(
    input_shape=X_train[0].shape,
    filters=64,
    kernel_size=(3,3),
    activation='relu'
))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2)
))
# Segunda convolución
model.add(Conv2D(
    filters=64,
    kernel_size=(3,3),
    activation='relu'
))
model.add(MaxPooling2D(
    pool_size=(2,2),
    strides=(2,2)
))
# Tercera Convolucion
model.add(Convolution2D(
    filters=128,
    kernel_size=(3,3),
    activation='relu'
))
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))
# Cuarta convolution.
model.add(Convolution2D(
    filters=128,
    kernel_size=(3,3),
    activation='relu'
))
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
))
# "Aplanamos" los resultados para pasar a la DNN
model.add(Flatten())
model.add(Dropout(0.5))
# Pasamos al la DNN para prediccion
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 128)      0

In [12]:
model.fit(
    X_train,
    y_train, 
    epochs=20,
    batch_size=32,
    verbose=1,
    validation_data=(X_test, y_test)
)

Epoch 1/20
79/79 [==============================] - 605s 8s/step - loss: 0.5831 - accuracy: 0.7194 - val_loss: 0.8769 - val_accuracy: 0.7419
Epoch 2/20
79/79 [==============================] - 609s 8s/step - loss: 0.0770 - accuracy: 0.9798 - val_loss: 1.0660 - val_accuracy: 0.7688
Epoch 3/20
79/79 [==============================] - 610s 8s/step - loss: 0.0217 - accuracy: 0.9940 - val_loss: 1.3593 - val_accuracy: 0.7473
Epoch 4/20
79/79 [==============================] - 615s 8s/step - loss: 0.0048 - accuracy: 0.9992 - val_loss: 1.8508 - val_accuracy: 0.7473
Epoch 5/20
79/79 [==============================] - 615s 8s/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 1.8558 - val_accuracy: 0.7634
Epoch 6/20
79/79 [==============================] - 625s 8s/step - loss: 0.0043 - accuracy: 0.9984 - val_loss: 2.2132 - val_accuracy: 0.7151
Epoch 7/20
79/79 [==============================] - 650s 8s/step - loss: 0.0320 - accuracy: 0.9913 - val_loss: 1.3163 - val_accuracy: 0.6452
Epoch 8/20
79

***

## Notas:

> Keras.Conv2D:
>   * filters: int(), Dimension de la salida de la convolución
>   * kernel_size: int / tuple(int, int) / list(int, int), matriz de convolución (n_px * n_px)

## Bibliografia:

   * Stack Overflow: https://stackoverflow.com
   * Elemento bibliografico 2

### Autor

   - **Name:** Elidas
   - **Email:** pyro.elidas@gmail.com
   - **Python version:** 3.9.1
   - **Date:** ${DATE}